# Module 5: Regime Prediction with Machine Learning - Part 2

In this part we will prepare the dataset for our recession forecasting problem. We will clean the data and perform feature selection to reduce the number of variables in the data.

## Table of Contents:
&nbsp;&nbsp;1. [Set Up Environment and Read Data](#1)

&nbsp;&nbsp;2. [Data Cleaning](#2)


## 1. Set Up Environment and Read Data <a id="1"></a>

In [4]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from statsmodels.tsa.stattools import adfuller #to check unit root in time series 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

import seaborn as sns #for correlation heatmap

import warnings
warnings.filterwarnings('ignore')

In [5]:
bigmacro=pd.read_csv("Macroeconomic_Variables.csv")
bigmacro=bigmacro.rename(columns={'sasdate':'Date'})
Recession_periods=pd.read_csv('Recession_Periods.csv')
bigmacro.insert(loc=1,column="Regime", value=Recession_periods['Regime'].values)
bigmacro.head()

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST,VXOCLSx
0,1/1/59,Normal,2437.296,2288.8,17.302,292258.8329,18235.77392,22.6248,23.4555,22.1893,...,11.358,2.13,2.45,2.04,NaN,274.9,6476.0,12298.0,84.2043,NaN
1,2/1/59,Normal,2446.902,2297.0,17.482,294429.5453,18369.56308,23.0679,23.7720,22.3816,...,11.375,2.14,2.46,2.05,NaN,276.0,6476.0,12298.0,83.5280,NaN
2,3/1/59,Normal,2462.689,2314.0,17.647,293425.3813,18523.05762,23.4002,23.9159,22.4914,...,11.395,2.15,2.45,2.07,NaN,277.4,6508.0,12349.0,81.6405,NaN
3,4/1/59,Normal,2478.744,2330.3,17.584,299331.6505,18534.46600,23.8987,24.2613,22.8210,...,11.436,2.16,2.47,2.08,NaN,278.1,6620.0,12484.0,81.8099,NaN
4,5/1/59,Normal,2493.228,2345.8,17.796,301372.9597,18679.66354,24.2587,24.4628,23.0407,...,11.454,2.17,2.48,2.08,95.3,280.1,6753.0,12646.0,80.7315,NaN


## 2. Data Cleaning <a id="2"></a>

We will follow the steps below to clean data and make it ready for feature selection process.

1. Remove the variables with missing observations
2. Add lags of the variables as additional features
3. Test stationarity of time series
4. Standardize the dataset

In [8]:
bigmacro.drop(['Date','Regime'],axis=1)

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,MZMSL,DTCOLNVHFNM,DTCTHFNM,INVEST
0,2437.296,2288.8,17.302,2.922588e+05,18235.77392,22.6248,23.4555,22.1893,32.4027,21.9673,...,56.918,17.791,11.358,2.13,2.45,2.04,274.9,6476.00,12298.00,84.2043
1,2446.902,2297.0,17.482,2.944295e+05,18369.56308,23.0679,23.7720,22.3816,32.6404,22.0826,...,56.951,17.798,11.375,2.14,2.46,2.05,276.0,6476.00,12298.00,83.5280
2,2462.689,2314.0,17.647,2.934254e+05,18523.05762,23.4002,23.9159,22.4914,32.6404,22.5150,...,57.022,17.785,11.395,2.15,2.45,2.07,277.4,6508.00,12349.00,81.6405
3,2478.744,2330.3,17.584,2.993317e+05,18534.46600,23.8987,24.2613,22.8210,33.1553,22.6592,...,57.080,17.796,11.436,2.16,2.47,2.08,278.1,6620.00,12484.00,81.8099
4,2493.228,2345.8,17.796,3.013730e+05,18679.66354,24.2587,24.4628,23.0407,33.3137,23.1204,...,57.175,17.777,11.454,2.17,2.48,2.08,280.1,6753.00,12646.00,80.7315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,16195.352,13448.0,117.056,1.487431e+06,505168.00000,107.4443,103.4982,102.2216,105.2565,119.7660,...,87.497,99.568,114.853,23.90,27.58,21.48,15514.8,303985.94,732446.08,3410.0486
714,16228.384,13474.2,117.450,1.490300e+06,508230.00000,107.8948,103.5866,102.4187,105.5223,119.1910,...,87.821,99.379,115.098,23.95,27.73,21.46,15542.1,306155.79,734076.57,3420.3257
715,16277.424,13516.2,117.804,1.497950e+06,507872.00000,108.8232,104.1772,103.1707,105.9314,121.7713,...,87.564,99.434,115.273,24.05,27.85,21.55,15561.9,308918.25,735979.69,3422.5104
716,16292.562,13520.2,117.799,1.502642e+06,506749.00000,109.0200,104.4810,103.6502,106.2259,123.2514,...,87.207,99.509,115.535,24.13,27.94,21.62,15589.7,307827.06,733943.95,3419.0700


In [6]:
#remove columns with missing observations
missing_colnames=[]
for i in bigmacro.drop(['Date','Regime'],axis=1):
    observations=len(bigmacro)-bigmacro[i].count()
    if (observations>10):
        print(i+':'+str(observations))
        missing_colnames.append(i)
 
bigmacro=bigmacro.drop(labels=missing_colnames, axis=1)

#rows with missing values
bigmacro=bigmacro.dropna(axis=0)

bigmacro.shape

PERMIT:13
PERMITNE:13
PERMITMW:13
PERMITS:13
PERMITW:13
ACOGNO:398
ANDENOx:110
TWEXMMTH:168
UMCSENTx:155
VXOCLSx:42


(718, 120)

In [9]:
len(bigmacro)

718

In [10]:
# Add lags
for col in bigmacro.drop(['Date', 'Regime'], axis=1):
    for n in [3,6,9,12,18]:
        bigmacro['{} {}M lag'.format(col, n)] = bigmacro[col].shift(n).ffill().values

# 1 month ahead prediction
bigmacro["Regime"]=bigmacro["Regime"].shift(-1)

bigmacro=bigmacro.dropna(axis=0)

In [11]:
bigmacro.columns

Index(['Date', 'Regime', 'RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx',
       'RETAILx', 'INDPRO', 'IPFPNSS', 'IPFINAL',
       ...
       'DTCTHFNM 3M lag', 'DTCTHFNM 6M lag', 'DTCTHFNM 9M lag',
       'DTCTHFNM 12M lag', 'DTCTHFNM 18M lag', 'INVEST 3M lag',
       'INVEST 6M lag', 'INVEST 9M lag', 'INVEST 12M lag', 'INVEST 18M lag'],
      dtype='object', length=710)

In [13]:
bigmacro

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DTCTHFNM 3M lag,DTCTHFNM 6M lag,DTCTHFNM 9M lag,DTCTHFNM 12M lag,DTCTHFNM 18M lag,INVEST 3M lag,INVEST 6M lag,INVEST 9M lag,INVEST 12M lag,INVEST 18M lag
18,7/1/60,Recession,2572.575,2417.0,18.262,2.930760e+05,18801.00720,23.8433,24.9232,23.6723,...,14413.00,14063.00,13804.00,13199.00,12298.00,71.4718,75.4515,76.2235,78.9984,84.2043
19,8/1/60,Recession,2568.401,2410.8,18.245,2.927061e+05,18865.30897,23.8156,24.8657,23.6723,...,14559.00,14144.00,13889.00,13471.00,12298.00,71.6858,73.3363,75.8793,77.3045,83.5280
20,9/1/60,Recession,2571.676,2411.5,18.332,2.974541e+05,18847.67784,23.5664,24.6642,23.5350,...,14816.00,14239.00,14058.00,13657.00,12349.00,71.1864,71.5296,76.1711,76.8926,81.6405
21,10/1/60,Recession,2579.283,2419.1,18.421,2.918478e+05,19013.61788,23.5387,24.8081,23.6448,...,15028.00,14413.00,14063.00,13804.00,12484.00,73.0020,71.4718,75.4515,76.2235,81.8099
22,11/1/60,Recession,2568.231,2405.2,18.357,2.889930e+05,18741.89106,23.2064,24.5491,23.3702,...,15208.00,14559.00,14144.00,13889.00,12646.00,73.0803,71.6858,73.3363,75.8793,80.7315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,5/1/18,Normal,16155.012,13412.8,116.680,1.485222e+06,503955.00000,106.7781,102.7694,101.2337,...,736073.92,742774.46,742096.43,739948.37,749064.90,3401.2403,3406.2524,3372.0672,3371.7914,3323.9290
713,6/1/18,Normal,16195.352,13448.0,117.056,1.487431e+06,505168.00000,107.4443,103.4982,102.2216,...,730477.90,742000.05,741795.94,743368.23,748123.97,3385.5601,3436.8783,3380.0367,3365.0940,3316.9344
714,7/1/18,Normal,16228.384,13474.2,117.450,1.490300e+06,508230.00000,107.8948,103.5866,102.4187,...,732599.77,739896.65,741392.57,743201.34,746218.48,3370.2407,3431.0044,3390.3759,3366.9032,3331.5539
715,8/1/18,Normal,16277.424,13516.2,117.804,1.497950e+06,507872.00000,108.8232,104.1772,103.1707,...,734677.52,736073.92,742774.46,742096.43,741101.40,3376.7011,3401.2403,3406.2524,3372.0672,3346.2017


Augmented Dickey-Fuller Test can be used to test for stationarity in macroeconomic time series variables. We will use `adfuller` function from `statsmodels` module in Python. More information about the function can be found __[here](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html)__.

In [14]:
#check stationarity
from statsmodels.tsa.stattools import adfuller #to check unit root in time series 
threshold=0.01 #significance level
for column in bigmacro.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacro[column])
    if result[1]>threshold:
        bigmacro[column]=bigmacro[column].diff()
bigmacro=bigmacro.dropna(axis=0)

In [15]:
threshold=0.01 #significance level
for column in bigmacro.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacro[column])
    if result[1]>threshold:
        bigmacro[column]=bigmacro[column].diff()
bigmacro=bigmacro.dropna(axis=0)

In [16]:
threshold=0.01 #significance level
for column in bigmacro.drop(['Date','Regime'], axis=1):
    result=adfuller(bigmacro[column])
    if result[1]>threshold:
        print(column)
bigmacro=bigmacro.dropna(axis=0)      

CES0600000008 6M lag


In [17]:
# Standardize
from sklearn.preprocessing import StandardScaler
features=bigmacro.drop(['Date','Regime'],axis=1)
col_names=features.columns

scaler=StandardScaler()
scaler.fit(features)
standardized_features=scaler.transform(features)
standardized_features.shape
df=pd.DataFrame(data=standardized_features,columns=col_names)
df.insert(loc=0,column="Date", value=bigmacro['Date'].values)
df.insert(loc=1,column='Regime', value=bigmacro['Regime'].values)
df.head()
df.shape

(697, 710)

In [18]:
df.to_csv("Dataset_Cleaned.csv", index=False)

In [19]:
df

,Date,Regime,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,...,DTCTHFNM 3M lag,DTCTHFNM 6M lag,DTCTHFNM 9M lag,DTCTHFNM 12M lag,DTCTHFNM 18M lag,INVEST 3M lag,INVEST 6M lag,INVEST 9M lag,INVEST 12M lag,INVEST 18M lag
0,9/1/60,Recession,-0.277761,-0.295013,-0.212118,0.401363,-0.321981,-0.764446,-0.632790,-0.465458,...,-0.076853,-0.092745,-0.087114,-0.085455,-0.099002,-0.454463,-0.566992,-0.395095,-0.450998,-0.089005
1,10/1/60,Recession,-0.204460,-0.161434,-0.204519,-0.978339,-0.239615,-0.308591,0.059474,-0.009134,...,-0.081327,-0.084890,-0.103429,-0.089335,-0.090643,-0.255545,-0.415804,-0.483302,-0.473520,0.147174
2,11/1/60,Recession,-0.520186,-0.577658,-0.785798,-0.611700,-0.435990,-0.935469,-0.748034,-0.719013,...,-0.084509,-0.087674,-0.095868,-0.095503,-0.087956,-0.404823,-0.392308,-0.605015,-0.445070,-0.091651
3,12/1/60,Recession,-0.531320,-0.581530,-1.397470,0.071269,-0.375495,-1.163499,-0.921200,-0.769798,...,-0.095048,-0.076637,-0.094476,-0.087146,-0.076214,-0.331993,-0.453979,-0.578110,-0.389378,-0.070555
4,1/1/61,Normal,0.054513,0.096044,-0.375484,-1.608684,-0.307555,-0.194576,-0.286457,-0.313288,...,-0.103002,-0.081112,-0.086616,-0.103462,-0.076910,-0.250758,-0.253854,-0.425585,-0.477942,0.167308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,5/1/18,Normal,-0.158774,-0.087869,0.961837,0.428800,2.458364,-2.071505,-3.897094,-4.542461,...,-0.482484,0.035214,-0.213850,0.107884,-0.230363,-2.969053,0.961673,0.029818,0.229887,-2.082950
693,6/1/18,Normal,0.349410,0.372881,0.885853,0.063037,0.230187,1.119480,1.231754,1.612471,...,-0.658796,-0.179194,-0.133821,0.236277,-0.197709,-1.758884,2.236719,0.274491,-1.001393,-0.345305
694,7/1/18,Normal,0.225752,0.198647,0.954238,0.150980,1.059808,0.675562,-0.051761,0.152085,...,0.108564,-0.311340,-0.144057,-0.120563,-0.293692,-1.727882,-0.918590,0.481157,-0.256506,1.560538
695,8/1/18,Normal,0.496621,0.504524,0.802270,0.788034,-0.474700,1.659097,0.954768,1.176830,...,0.104177,-0.482299,0.033552,-0.213885,-0.613278,0.143563,-2.983834,0.964076,0.037260,0.000571
